In [2]:
%%javascript

window.load_remote_theme = true
var theme_js = "https://odhk.github.io/hyrule_theme/custom.js";

window.load_local_theme = function(){
    var hostname = document.location.hostname
    return ((hostname == "localhost" || hostname == '127.0.0.1') && !load_remote_theme)
}

var url = load_local_theme() ? document.location.origin + "/files/theme/custom.js" : theme_js

$.getScript(url)

<IPython.core.display.Javascript object>

In [7]:
import numpy as np
import pandas as pd
import gc
import lightgbm as lgb
import keras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Lambda, Embedding,constraints,\
Dropout, Activation,GRU,Bidirectional,Subtract, Permute, TimeDistributed, Reshape
from keras.layers import Conv1D,Conv2D,MaxPooling2D,GlobalAveragePooling1D,GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers import CuDNNGRU, CuDNNLSTM, SpatialDropout1D,Layer, initializers, regularizers
from keras.layers.merge import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract
from keras.models import Model
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras import backend as K
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
import gc
import tensorflow as tf
from keras.activations import softmax

from keras.utils import plot_model
from keras.layers import *
from keras.applications.imagenet_utils import _obtain_input_shape
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.special import erfinv

import gensim
from gensim.models import word2vec
from gensim.models.word2vec import LineSentence
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

label = train['class']

char_embedding_index = {}
f = open('word_embedding_300dim_new.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0].lower()
    embeddings = np.asarray(values[1:], dtype = 'float32')
    char_embedding_index[word] = embeddings
f.close()

len(char_embedding_index)

api_embedding_file_path = 'word_embedding_300dim_new.txt'
embedding_dim = 300
max_nb_api = len(char_embedding_index) + 1
max_sequence_length_api =1500

all_word = list(train['article'].values) + list(test['article'].values)

all_word_list = []
for i in list(all_word):
    all_word_list.append(i.split(' '))

len(all_word_list)

train['article'].apply(lambda x:len(x.split(' '))).describe()

data = pd.DataFrame()

gc.collect()

tokenizer_api = Tokenizer()

tokenizer_api.fit_on_texts(all_word_list)

api_seq = tokenizer_api.texts_to_sequences(all_word_list)

data['article'] = pad_sequences(api_seq, maxlen = max_sequence_length_api, padding='post').tolist()

api_index = tokenizer_api.word_index
print('Found %s unique tokens' % len(api_index))
nb_words = min(max_nb_api, len(char_embedding_index)+1)
api_embedding_matrix = np.zeros((nb_words, embedding_dim))

for word, i in tqdm(api_index.items()):
    embedding_vector = char_embedding_index.get(word)
    if embedding_vector is not None:
        api_embedding_matrix[i] = embedding_vector
    else:
        print('no char%s'%word)

data['id'] = train['id']

train_FE = data.iloc[:len(label),:]
test_FE = data.iloc[len(label):,:]

os.environ["CUDA_VISIBLE_DEVICES"] = "6, 7"

def get_input(df):
    
    _input = [np.array(df.article.values.tolist())]
    
    
    return _input 

16053it [00:01, 12752.21it/s]
100%|██████████| 16052/16052 [00:00<00:00, 417345.23it/s]

Found 16052 unique tokens


In [8]:
def attention(x):
    
    q1, q2 = x[0], x[1]
    
    #compute match matrix euclidean match score
    match_matrix = 1. + K.sqrt(
        -2 * K.batch_dot(q1, q2, axes=[2, 2]) +
        K.expand_dims(K.sum(K.square(q1), axis=2), 2) +
        K.expand_dims(K.sum(K.square(q2), axis=2), 1)
    )
    
    match_matrix = K.maximum(match_matrix, K.epsilon())
    match_matrix = 1. / match_matrix
    #compute attention output
    q1_new = K.batch_dot(K.softmax(match_matrix, axis = 1), q2)
    match_matrix_T = Permute((2, 1))(match_matrix)
    q2_new = K.batch_dot(K.softmax(match_matrix_T, axis = 1), q1)
    
    return [q1_new, q2_new]

In [15]:
def Globel_attention_DR_Stack_RNN():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    
    #RNN
    char_rnn_layer_1 = Bidirectional(CuDNNLSTM(150, 
                               return_sequences=True))
    char_rnn_layer_2 = Bidirectional(CuDNNLSTM(150, 
                               return_sequences=True))
    
    char_embedding = api_embedding_layer(char_input)
    char_embedding = SpatialDropout1D(0.2)(char_embedding)
    
    #双层BiLSTM
    char_rnn_1= char_rnn_layer_1(char_embedding)
    char_rnn_2= char_rnn_layer_2(char_rnn_1)
    
    #max avg 
    char_att_maxpooling_1 = GlobalMaxPooling1D()(char_rnn_1)
    char_att_avgpooling_1 = GlobalAveragePooling1D()(char_rnn_1)
    
    char_att_maxpooling_2 = GlobalMaxPooling1D()(char_rnn_2)
    char_att_avgpooling_2 = GlobalAveragePooling1D()(char_rnn_2)
    
    #batchnormalization
    char_embedding = BatchNormalization()(char_embedding)
    char_rnn_2 = BatchNormalization()(char_rnn_2)
    
    #globel attention information
    r = Lambda(attention, name = 'attention')([char_embedding, char_rnn_2])
    
    attention_map_left = r[0]
    attention_map_right = r[1]
    
    att_maxpooling_left = GlobalMaxPooling1D()(attention_map_left)
    att_maxpooling_right = GlobalMaxPooling1D()(attention_map_right)
    
    att_avgpooling_left = GlobalAveragePooling1D()(attention_map_left)
    att_avgpooling_right = GlobalAveragePooling1D()(attention_map_right)
   
    max_sub_char_abs_attention = Lambda(lambda x:K.abs(x[0] - x[1]))([att_maxpooling_left, att_maxpooling_right])
    max_multi_char_attention = Lambda(lambda x: x[0] * x[1])([att_maxpooling_left, att_maxpooling_right])

    avg_sub_char_abs_attention = Lambda(lambda x:K.abs(x[0] - x[1]))([att_avgpooling_left, att_avgpooling_right])
    avg_multi_char_attention = Lambda(lambda x: x[0] * x[1])([att_avgpooling_left, att_avgpooling_right])

    merged = concatenate([char_att_maxpooling_1, char_att_avgpooling_1, \
                          char_att_maxpooling_2, char_att_avgpooling_2,\
                         max_sub_char_abs_attention, max_multi_char_attention, \
                         avg_sub_char_abs_attention, avg_multi_char_attention, \
                         att_maxpooling_left, att_maxpooling_right, \
                         att_avgpooling_left, att_avgpooling_right])
    merged = Dropout(0.4)(merged)
    merged = BatchNormalization()(merged)
    merged = Dense(512, activation='relu')(merged)
    merged = Dropout(0.1)(merged)
    merged = BatchNormalization()(merged) 
    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model

In [10]:
meta_train = pd.DataFrame(np.array([None]*train_FE.shape[0]*19).reshape(train_FE.shape[0], 19))
meta_train.columns = ['prob' + str(x) for x in range(19)]
meta_test = pd.DataFrame()

label = train['class']

label = label - 1

categorical_labels = keras.utils.np_utils.to_categorical(label, num_classes=19)

index = pd.read_csv('fold.csv')
index['id'] = train['id']

In [11]:
del test, all_word
gc.collect()

41

In [ ]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64

    model = Globel_attention_DR_Stack_RNN()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=10, batch_size=BATCH_SIZE, shuffle=True)#callbacks=callbacks

    
    pred = pd.DataFrame(model.predict(X_val,batch_size=128,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=128,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/10
92040/92040 [==============================] - 1083s 12ms/step - loss: 1.4212 - acc: 0.5843 - val_loss: 1.1379 - val_acc: 0.6685
Epoch 2/10
92040/92040 [==============================] - 1082s 12ms/step - loss: 1.0931 - acc: 0.6754 - val_loss: 1.0125 - val_acc: 0.7039
Epoch 3/10
92040/92040 [==============================] - 1085s 12ms/step - loss: 0.8367 - acc: 0.7476 - val_loss: 0.8689 - val_acc: 0.7401
Epoch 8/10
92040/92040 [==============================] - 1086s 12ms/step - loss: 0.8025 - acc: 0.7573 - val_loss: 0.8330 - val_acc: 0.7513
Epoch 9/10
92040/92040 [==============================] - 1085s 12ms/step - loss: 0.7769 - acc: 0.7639 - val_loss: 0.8293 - val_acc: 0.7525
Epoch 10/10
102277/102277 [==============================] - 332s 3ms/step
*******1*******
Train on 92043 samples, validate on 10234 samples
Epoch 1/10
92043/92043 [==============================] - 1094s 12ms/step - loss: 1.4156 - ac

In [17]:
meta_train.columns = ['meta_rnn_v7_dim_300_' + x for x in meta_train.columns]

p = pd.DataFrame()

for i in range(19):
    p[i] = meta_test[i].mean(axis = 1)

p.columns = meta_train.columns

In [18]:
meta_train.shape, p.shape

((102277, 19), (102277, 19))

In [19]:
meta_train.to_csv('/home/libo/daguan/stack/train_meta_rnn_v7_dim_300.csv', index = None ,encoding = 'utf-8')

p.to_csv('/home/libo/daguan/stack/test_meta_rnn_v7_dim_300.csv', index = None ,encoding = 'utf-8')